# MIKH Model: Large Amplitude Oscillatory Shear (LAOS)

## Learning Objectives

1. Fit the **MIKH** model to LAOS data (nonlinear oscillatory response)
2. Analyze **Lissajous figures** (stress vs strain loops)
3. Extract **harmonic content** from the stress response
4. Understand the role of **plasticity** and **thixotropy** in nonlinear oscillatory response
5. Compare response at different frequencies

## Prerequisites

- NB01: MIKH Flow Curve (parameter understanding)
- NB05: MIKH SAOS (linear viscoelastic baseline)

## Runtime

- Fast demo: ~4-5 minutes
- Full run: ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from scipy.fft import rfft, rfftfreq

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.ikh import MIKH

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from ikh_tutorial_utils import (
    load_pnas_laos,
    save_ikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_mikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: LAOS in Thixotropic Materials

Large Amplitude Oscillatory Shear (LAOS) probes the **nonlinear** viscoelastic and viscoplastic response.

### Applied Deformation

$$
\gamma(t) = \gamma_0 \sin(\omega t)
$$

where $\gamma_0 > \gamma_{yield}$ (large enough to induce plastic flow).

### Nonlinear Stress Response

The stress response contains **odd harmonics**:
$$
\sigma(t) = \sum_{n=1,3,5,...} [\sigma_n' \sin(n\omega t) + \sigma_n'' \cos(n\omega t)]
$$

### Key Nonlinear Features in MIKH

1. **Plastic yielding**: Distorts sinusoidal stress response
2. **Kinematic hardening**: Creates asymmetry in strain-stress loops
3. **Thixotropic restructuring**: Amplitude/frequency-dependent structure

### Lissajous Figures

Plotting $\sigma$ vs $\gamma$ reveals:
- **Ellipse**: Linear viscoelastic (SAOS)
- **Distorted ellipse**: Nonlinear viscoelastic
- **Rectangular**: Yield stress dominated

### Harmonic Analysis

The **third harmonic ratio** $I_3/I_1$ quantifies nonlinearity:
- $I_3/I_1 \to 0$: Linear response
- $I_3/I_1 > 0$: Nonlinear (strain stiffening or softening)

## 3. Load Data

We load LAOS data from the PNAS Digital Rheometer Twin dataset at $\omega = 1$ rad/s.

In [ ]:
# Load LAOS data at omega = 1 rad/s, medium amplitude
omega = 1.0
strain_amp_idx = 8  # Medium-large amplitude

t_data, strain_data, stress_data = load_pnas_laos(omega=omega, strain_amplitude_index=strain_amp_idx)

# Estimate strain amplitude from data
gamma_0 = np.max(np.abs(strain_data))

print(f"LAOS data loaded:")
print(f"  omega = {omega} rad/s")
print(f"  gamma_0 = {gamma_0:.4f} (estimated)")
print(f"  Points: {len(t_data)}")
print(f"  Time range: [{t_data.min():.3f}, {t_data.max():.2f}] s")

In [ ]:
# Plot raw LAOS data
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Time series - strain
axes[0].plot(t_data, strain_data, "k-", lw=1)
axes[0].set_xlabel("Time [s]", fontsize=11)
axes[0].set_ylabel("Strain [-]", fontsize=11)
axes[0].set_title("Strain Input", fontsize=12)
axes[0].grid(True, alpha=0.3)

# Time series - stress
axes[1].plot(t_data, stress_data, "b-", lw=1)
axes[1].set_xlabel("Time [s]", fontsize=11)
axes[1].set_ylabel("Stress [Pa]", fontsize=11)
axes[1].set_title("Stress Response", fontsize=12)
axes[1].grid(True, alpha=0.3)

# Lissajous figure
axes[2].plot(strain_data, stress_data, "b-", lw=0.5, alpha=0.7)
axes[2].set_xlabel("Strain [-]", fontsize=11)
axes[2].set_ylabel("Stress [Pa]", fontsize=11)
axes[2].set_title("Lissajous Figure (Raw)", fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit MIKH model to LAOS data
model = MIKH()
param_names = get_mikh_param_names()

print(f"Fitting MIKH to LAOS data (gamma_0 = {gamma_0:.3f}, omega = {omega} rad/s)")
t0 = time.time()
model.fit(t_data, stress_data, test_mode="laos", gamma_0=gamma_0, omega=omega)
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Predict and compute fit quality
stress_pred = model.predict_laos(t_data, gamma_0=gamma_0, omega=omega)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

In [ ]:
# Plot fit: time series
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(t_data, stress_data, "k-", lw=1, alpha=0.7, label="Data")
ax.plot(t_data, stress_pred, "-", lw=2, color="C0", label="MIKH fit")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title(f"LAOS Fit: Time Series (R$^2$ = {metrics['R2']:.4f})", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Lissajous comparison
strain_pred = gamma_0 * np.sin(omega * np.array(t_data))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Data Lissajous
ax1.plot(strain_data, stress_data, "k-", lw=1, alpha=0.7)
ax1.set_xlabel("Strain [-]", fontsize=12)
ax1.set_ylabel("Stress [Pa]", fontsize=12)
ax1.set_title("Lissajous: Data", fontsize=13)
ax1.grid(True, alpha=0.3)

# Model Lissajous
ax2.plot(strain_pred, stress_pred, "-", lw=2, color="C0")
ax2.set_xlabel("Strain [-]", fontsize=12)
ax2.set_ylabel("Stress [Pa]", fontsize=12)
ax2.set_title("Lissajous: MIKH Model", fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Harmonic Analysis

In [ ]:
# Extract harmonics via FFT
def extract_harmonics(t, y, omega, n_harmonics=5):
    """Extract harmonic amplitudes from oscillatory signal."""
    n = len(y)
    dt = np.mean(np.diff(t))
    
    # FFT
    Y = rfft(y)
    freqs = rfftfreq(n, dt)
    
    # Find fundamental frequency index
    f0 = omega / (2 * np.pi)
    idx_f0 = np.argmin(np.abs(freqs - f0))
    
    # Extract harmonic amplitudes
    harmonics = {}
    for h in range(1, n_harmonics + 1):
        idx = idx_f0 * h
        if idx < len(Y):
            harmonics[h] = 2 * np.abs(Y[idx]) / n
        else:
            harmonics[h] = 0.0
    
    return harmonics

# Extract harmonics from data and model
harmonics_data = extract_harmonics(t_data, stress_data, omega)
harmonics_model = extract_harmonics(t_data, np.array(stress_pred), omega)

print("Harmonic Analysis:")
print(f"{'Harmonic':<10} {'Data [Pa]':>12} {'Model [Pa]':>12}")
print("-" * 36)
for h in range(1, 6):
    print(f"  I_{h:<6}    {harmonics_data.get(h, 0):12.3f}   {harmonics_model.get(h, 0):12.3f}")

# Third harmonic ratio (nonlinearity measure)
I3_I1_data = harmonics_data.get(3, 0) / harmonics_data.get(1, 1)
I3_I1_model = harmonics_model.get(3, 0) / harmonics_model.get(1, 1)
print(f"\nThird harmonic ratio I_3/I_1:")
print(f"  Data:  {I3_I1_data:.4f}")
print(f"  Model: {I3_I1_model:.4f}")

In [ ]:
# Harmonic spectrum plot
fig, ax = plt.subplots(figsize=(10, 5))

harmonics = [1, 3, 5, 7, 9]
data_vals = [harmonics_data.get(h, 0) for h in harmonics]
model_vals = [harmonics_model.get(h, 0) for h in harmonics]

x = np.arange(len(harmonics))
width = 0.35

ax.bar(x - width/2, data_vals, width, label="Data", color="gray", alpha=0.7)
ax.bar(x + width/2, model_vals, width, label="MIKH", color="C0", alpha=0.7)

ax.set_ylabel("Amplitude [Pa]", fontsize=12)
ax.set_xlabel("Harmonic number", fontsize=12)
ax.set_title(f"Harmonic Spectrum ($\\gamma_0$ = {gamma_0:.3f}, $\\omega$ = {omega} rad/s)", fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels([f"$I_{h}$" for h in harmonics])
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Multi-Frequency Analysis

In [ ]:
# Load LAOS at multiple frequencies
omega_values = [1.0, 3.0, 5.0]
laos_datasets = {}

for w in omega_values:
    t, strain, stress = load_pnas_laos(omega=w, strain_amplitude_index=strain_amp_idx)
    laos_datasets[w] = {"time": t, "strain": strain, "stress": stress}
    print(f"omega = {w} rad/s: {len(t)} points, gamma_0 ~ {np.max(np.abs(strain)):.3f}")

In [ ]:
# Compare Lissajous figures at different frequencies
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, w in enumerate(omega_values):
    d = laos_datasets[w]
    
    # Data
    axes[i].plot(d["strain"], d["stress"], "k-", lw=0.5, alpha=0.7, label="Data")
    
    # Model prediction
    gamma_0_w = np.max(np.abs(d["strain"]))
    stress_pred_w = model.predict_laos(d["time"], gamma_0=gamma_0_w, omega=w)
    strain_pred_w = gamma_0_w * np.sin(w * np.array(d["time"]))
    axes[i].plot(strain_pred_w, stress_pred_w, "-", lw=1.5, color="C0", alpha=0.8, label="MIKH")
    
    axes[i].set_xlabel("Strain [-]", fontsize=11)
    axes[i].set_ylabel("Stress [Pa]", fontsize=11)
    axes[i].set_title(f"$\\omega$ = {w} rad/s", fontsize=12)
    axes[i].grid(True, alpha=0.3)
    if i == 0:
        axes[i].legend(fontsize=9)

plt.suptitle("Lissajous Figures at Different Frequencies", fontsize=13)
plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Bayesian Inference

In [ ]:
# Bayesian inference
initial_values = {name: model.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    t_data,
    stress_data,
    test_mode="laos",
    gamma_0=gamma_0,
    omega=omega,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots
idata = result.to_inference_data()
laos_params = ["G", "C", "gamma_dyn", "sigma_y0", "delta_sigma_y"]
axes = az.plot_trace(idata, var_names=laos_params, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (LAOS-Sensitive Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 8. Physical Interpretation

### Lissajous Shape Analysis

The shape of the Lissajous figure reveals the material character:

- **Narrow ellipse**: Elastic-dominated
- **Wide ellipse**: Viscous-dominated
- **Rectangular**: Yield stress dominated (rate-independent plasticity)
- **Parallelogram**: Kinematic hardening present

### Nonlinearity Sources in MIKH

1. **Plastic flow**: When $|\sigma - \alpha| > \sigma_y$, creates distortion
2. **Kinematic hardening**: Backstress $\alpha$ shifts yield surface during cycle
3. **Thixotropic dynamics**: Structure varies within cycle at low frequencies

### Frequency Dependence

- **Low frequency** ($\omega \ll 1/\tau_{thix}$): Full thixotropic restructuring within cycle
- **High frequency** ($\omega \gg 1/\tau_{thix}$): Structure frozen, pure elastoplastic response

## 9. Save Results

In [ ]:
# Save results
save_ikh_results(model, result, "mikh", "laos", param_names)

## Key Takeaways

1. **LAOS** probes the nonlinear mechanical response of MIKH materials beyond the yield stress

2. **Lissajous figures** reveal the interplay of elasticity, viscosity, and plasticity

3. **Harmonic analysis** quantifies nonlinearity through $I_3/I_1$ and higher harmonics

4. **Kinematic hardening** ($C$, $\gamma_{dyn}$) creates asymmetry in the stress-strain loops

5. **Frequency dependence**: Low $\omega$ allows thixotropic restructuring; high $\omega$ freezes structure

6. The MIKH model captures the essential LAOS features through its elastoplastic + thixotropic framework

### Next Steps (MLIKH Notebooks)

- **NB07**: MLIKH Flow Curve (multi-mode thixotropic yield)
- **NB08**: MLIKH Startup (richer overshoot dynamics)
- **NB09-12**: MLIKH for remaining protocols